In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from utils.data_loader import load_as_maps
from models.unet import build_unet
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from optuna.integration import TFKerasPruningCallback
from datetime import datetime

In [ ]:
# load data
features, targets = load_as_maps(start_year=1958, end_year=2018, datasets=["exp1"])

# split data
X_train = features[:int(0.8 * len(features))]
Y_train = targets[:int(0.8 * len(targets))]
X_val = features[int(0.8 * len(features)):int(0.9 * len(features))]
Y_val = targets[int(0.8 * len(targets)):int(0.9 * len(targets))]
X_test = features[int(0.9 * len(features)):]
Y_test = targets[int(0.9 * len(targets)):]

# scale data
scaler = MinMaxScaler()
n_samples, h, w, n_features = X_train.shape
X_train_flat = X_train.reshape(-1,n_features)
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_train = X_train_scaled_flat.reshape(n_samples, h, w, n_features)


n_samples, h, w, n_features = X_val.shape
X_val_flat = X_val.reshape(-1,n_features)
X_val_scaled_flat = scaler.transform(X_val_flat)
X_val = X_val_scaled_flat.reshape(n_samples, h, w, n_features)

n_samples, h, w, n_features = X_test.shape
X_test_flat = X_test.reshape(-1,n_features)
X_test_scaled_flat = scaler.transform(X_test_flat)
X_test = X_test_scaled_flat.reshape(n_samples, h, w, n_features)

In [ ]:
tf.get_logger().setLevel('ERROR')  # quieter logs

# Optional: determinism
tf.random.set_seed(42)
np.random.seed(42)

# ---------- define the objective ----------
def build_model(lr, optimizer_name, base_filters, kernel_size, dropout_rate):
    
    model = build_unet((167, 360, 13),base_filters,kernel_size,dropout_rate)

    # ---- choose optimizer ----
    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer_name == "adamw":
        optimizer = tf.keras.optimizers.AdamW(learning_rate=lr)
    elif optimizer_name == "nadam":
        optimizer = tf.keras.optimizers.Nadam(learning_rate=lr)
    elif optimizer_name == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    elif optimizer_name == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    return model

def objective(trial: optuna.Trial):
    # ----- search space -----
    batch_size = trial.suggest_categorical("batch_size", [1, 2, 4, 8])
    lr = trial.suggest_categorical("lr", [0.0001, 0.0002, 0.0005, 0.001])
    kernel_size = trial.suggest_categorical("kernel_size", [3, 4])
    dropout_rate = trial.suggest_categorical("dropout_rate", [0, 0.05, 0.1, 0.2])
    base_filters = trial.suggest_categorical("base_filters", [32,64])
    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "adamw", "nadam", "rmsprop"])

    model = build_model(lr, optimizer_name, base_filters, kernel_size, dropout_rate)

    # ----- callbacks (with pruning) -----
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True
    )
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7
    )
    pruning_cb = TFKerasPruningCallback(trial, monitor='val_loss')

    # If your data are NumPy arrays X_train/Y_train, X_val/Y_val already exist in scope
    history = model.fit(
        X_train, Y_train,
        validation_data=(X_val, Y_val),
        epochs=40,
        batch_size=batch_size,
        verbose=0,
        callbacks=[early_stopping, lr_scheduler, pruning_cb],
        shuffle=True
    )

    # Return the best validation loss from training
    val_losses = history.history['val_loss']
    return float(np.min(val_losses))

# ---------- run the study ----------
study = optuna.create_study(
    direction="minimize",
    sampler=TPESampler(seed=42),
    pruner=MedianPruner(n_warmup_steps=10)
)
# Adjust n_trials to your budget
study.optimize(objective, n_trials=100, gc_after_trial=True)

print("Best trial:")
print("  value (val_loss):", study.best_trial.value)
print("  params:", study.best_trial.params)


[I 2025-09-24 16:51:56,896] A new study created in memory with name: no-name-8ec02b1f-245c-439b-96c6-4e15a5759dde


19/19 ━━━━━━━━━━━━━━━━━━━━ 118s 6s/step - loss: 2.0554 - mae: 0.9131 - val_loss: 89592176.0000 - val_mae: 7220.2891 - learning_rate: 0.0010


[I 2025-09-24 16:53:55,197] Trial 0 finished with value: 89592176.0 and parameters: {'batch_size': 2, 'lr': 0.001, 'kernel_size': 7, 'dropout_rate': 0, 'base_filters': 64, 'optimizer': 'nadam'}. Best is trial 0 with value: 89592176.0.


5/5 ━━━━━━━━━━━━━━━━━━━━ 37s 7s/step - loss: 3.1316 - mae: 1.2270 - val_loss: 5530622407737344.0000 - val_mae: 58151008.0000 - learning_rate: 0.0020


[I 2025-09-24 16:54:32,876] Trial 1 finished with value: 5530622407737344.0 and parameters: {'batch_size': 8, 'lr': 0.002, 'kernel_size': 7, 'dropout_rate': 0, 'base_filters': 32, 'optimizer': 'rmsprop'}. Best is trial 0 with value: 89592176.0.


...

10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - loss: 1.4303 - mae: 0.8228 - val_loss: 1.1331 - val_mae: 0.6260 - learning_rate: 5.0000e-04


[I 2025-09-24 16:59:55,634] Trial 8 finished with value: 1.133068561553955 and parameters: {'batch_size': 4, 'lr': 0.0005, 'kernel_size': 5, 'dropout_rate': 0.2, 'base_filters': 32, 'optimizer': 'rmsprop'}. Best is trial 4 with value: 1.1235020160675049.


38/38 ━━━━━━━━━━━━━━━━━━━━ 12s 269ms/step - loss: 0.8739 - mae: 0.6170 - val_loss: 1.1110 - val_mae: 0.6356 - learning_rate: 2.0000e-04


[I 2025-09-24 17:00:08,491] Trial 9 finished with value: 1.1109514236450195 and parameters: {'batch_size': 1, 'lr': 0.0002, 'kernel_size': 3, 'dropout_rate': 0.05, 'base_filters': 32, 'optimizer': 'rmsprop'}. Best is trial 9 with value: 1.1109514236450195.


Best trial:
  value (val_loss): 1.1109514236450195
  params: {'batch_size': 1, 'lr': 0.0002, 'kernel_size': 3, 'dropout_rate': 0.05, 'base_filters': 32, 'optimizer': 'rmsprop'}


In [ ]:
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M')
folder_path = "../../outputs/optuna/u-net"

df = study.trials_dataframe()

# sort descending by 'value'
df = df.sort_values(by="value", ascending=True)

# round the 'value' column to 4 decimals
df["value"] = df["value"].round(4)

# save to CSV
df.to_csv(folder_path + "/trials_" + timestamp + ".csv", index=False)